In [7]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import eig, expm
from scipy.optimize import minimize
from functools import reduce 
from xmps.spin import U4

# state:
"""
                   0      0      j
                   | -U2- |      |
A[i, σ1,σ2, j] =   |      | -U1- |
                   |      |      |
                   i      σ1      σ2
"""

# alternate state:
"""
                   i      0      0
                   |      | -U2- |
A[i, σ1,σ2, j] =   | -U1- |      |
                   |      |      |
                   σ1     σ2     j
"""

Z = np.array([
    [1,0],
    [0,-1]
])

X = np.array([
    [0,1],
    [1,0]
])

I = np.eye(2)

def tensor(tensors):
    return reduce(lambda t1,t2: np.kron(t1,t2), tensors)



In [8]:
def state(U2, U1):
    """
    A[i,σ1,σ2,j] = Σα U1[σ1,σ2,α,j]U2[i,α,0,0]
    """
    return np.transpose(np.tensordot(
        U1.reshape(2,2,2,2),
        U2[:,0].reshape(2,2),
        (2,1)
    ),(3,0,1,2)).reshape(2,4,2)
    
def check_state():
    U2 = np.eye(4)
    U1 = np.eye(4)
    
    s = state(U2,U1)
        
    assert s[0,0,0] == 1
    assert s[0,1,0] == 0
    assert s[0,0,1] == 0
    assert s[0,2,1] == 0
    assert s[0,2,0] == 0
    assert s[1,0,0] == 0
    assert s[1,1,1] == 0
    assert s[0,1,1] == 1

def alternate_state(U2, U1):
    """
    A[i,σ1,σ2,j] = Σα U1[σ1,σ2,i,α]U2[α,j,0,0]
    """
    
    return np.transpose(np.tensordot(
        U1.reshape(2,2,2,2),
        U2[:,0].reshape(2,2),
        (3,0)
    ),(2,0,1,3)).reshape(2,4,2)

def check_alternate_state():
    U2 = np.eye(4)
    U1 = np.eye(4)
    
    s = alternate_state(U2, U1)
    
    assert s[0,0,0] == 1
    assert s[1,2,0] == 1
    assert np.sum(s,(0,1,2)) == 2

check_state()
check_alternate_state()

In [9]:
def invert_long(U2,U1):
    """
    Aⁱ[i',σ1,σ2, j'] = U2ⁱ[0,0,i',α]U1[α,j',σ1,σ2]
    """
    return np.transpose(np.tensordot(
        U2.conj().T[0,:].reshape(2,2),
        U1.conj().T.reshape(2,2,2,2),
        (1,0)
    ),(0,2,3,1)).reshape(2,4,2)

def invert(s):
    return s.conj()


def check_inverse(s):

    U1 = unitary_group.rvs(4)
    U2 = unitary_group.rvs(4)

    s  = state(U2,U1)
    s_ = invert(s)
    
    ss_ = np.tensordot(s_.reshape(8,2),s.reshape(8,2),(0,0))
    assert np.allclose(ss_, np.eye(2))
    
    ss_ = np.transpose(np.tensordot(s_,s,(1,1)),(0,2,1,3)).reshape(4,4)
    print(np.round(ss_,3))
    assert np.allclose(ss_[:,1], np.array([0,0,0,0]))
    assert np.allclose(ss_[:,2], np.array([0,0,0,0]))



In [11]:
def chain(s1,s2):
    return np.tensordot(s1,s2,(-1,0))

def single_transfer_matrix(s, s̄):
    s̄ = s̄.conj()
    ss̄ = np.transpose(np.tensordot(s̄,s,(1,1)),(0,2,1,3)).reshape(4,4)
    return ss̄

def double_transfer_matrix(s,s̄):
    ss = chain(s,s).reshape(2,16,2)
    s̄s̄ = chain(s̄,s̄).conj().reshape(2,16,2)
    sss̄s̄ = np.transpose(np.tensordot(s̄s̄, ss, (1,1)), (0,2,1,3)).reshape(4,4)
    return sss̄s̄

    
    
    

In [12]:
U1 = unitary_group.rvs(4)
U2 = unitary_group.rvs(4)

s  = state(U2,U1)

η, v = eig(single_transfer_matrix(s,s))
print("Right Environment:")
print(np.round(η,3))
print(np.round(v,3))

print("\n")
print("Left Environment")
η, v = eig(single_transfer_matrix(s,s).T)
print(np.round(η,3))
print(np.round(v,3))

Right Environment:
[ 1.+0.j -0.+0.j  0.-0.j -0.+0.j]
[[ 0.798+0.j     0.092+0.448j -0.01 +0.218j  0.108-0.44j ]
 [-0.073-0.358j -0.496+0.212j  0.673+0.j     0.543+0.j   ]
 [-0.073+0.358j  0.539+0.j    -0.67 -0.062j -0.482-0.251j]
 [ 0.31 +0.j    -0.092-0.448j  0.01 -0.218j -0.108+0.44j ]]


Left Environment
[ 1.-0.j -0.+0.j -0.-0.j  0.+0.j]
[[ 0.707+0.j    -0.277+0.008j  0.302-0.262j -0.05 -0.064j]
 [ 0.   +0.j     0.474+0.23j   0.362-0.184j -0.169-0.35j ]
 [ 0.   -0.j     0.495-0.04j   0.736+0.j    -0.032+0.323j]
 [ 0.707+0.j     0.632+0.j    -0.306+0.201j  0.859+0.j   ]]


In [100]:
np.round(single_transfer_matrix(s,s),3)

array([[ 0.676+0.j   , -0.   -0.j   , -0.   +0.j   ,  0.676+0.j   ],
       [-0.193+0.418j,  0.   -0.j   ,  0.   -0.j   , -0.193+0.418j],
       [-0.193-0.418j,  0.   +0.j   ,  0.   +0.j   , -0.193-0.418j],
       [ 0.324+0.j   , -0.   -0.j   , -0.   +0.j   ,  0.324+0.j   ]])

In [13]:
U2 = np.eye(4)
U1 = np.eye(4)

s = alternate_state(U2, U1)
ss = single_transfer_matrix(s,s)
print("Identity Alternate Transfer Matrix:")
print(ss)
print("\n")
 
U2 = unitary_group.rvs(4)
U1 = unitary_group.rvs(4)

s = alternate_state(U2,U1)
ss = single_transfer_matrix(s, s)

η, v = eig(ss)
print("Right Environment Adjustment:")
print(np.round(η,3))
print(np.round(v,3))

Identity Alternate Transfer Matrix:
[[1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 0.]]


Right Environment Adjustment:
[ 1.-0.j -0.-0.j -0.+0.j -0.+0.j]
[[ 0.707+0.j     0.663+0.j     0.616+0.j    -0.197-0.645j]
 [-0.   +0.j     0.34 -0.107j  0.158+0.591j  0.688+0.j   ]
 [-0.   -0.j    -0.031-0.578j  0.416+0.088j -0.145-0.105j]
 [ 0.707+0.j    -0.278-0.142j -0.238+0.094j  0.172+0.106j]]


In [3]:
def Hamiltonian(J, g):
    ZZ = 0.5*(tensor([Z,Z,I,I]) + tensor([I,I,Z,Z])) + tensor([I,Z,Z,I])
    XX = 0.5*(tensor([X,I,I,I]) + tensor([I,X,I,I]) + tensor([I,I,X,I]) + tensor([I,I,I,X]))
    return J*ZZ + g*XX

Hamiltonian(1,1).shape

NameError: name 'tensor' is not defined

In [14]:
U2 = unitary_group.rvs(4)
U1 = unitary_group.rvs(4)
s = state(U2, U1)

def exact_right_environment(s, s̄):
    dt= single_transfer_matrix(s,s̄)
    η, v = eig(dt)
    vr = v[:,np.argmax(np.abs(η))].reshape(2,2)
    return vr


#################################
# Exact environment calculation
#################################
exact_r = exact_right_environment(s, s)
print("Exact environment:\n")
print(np.round(exact_r,3))
print("Norm: ", np.linalg.norm(exact_r))
#################################
print("\n")
def approx_env(U2, Ū2):
    return np.tensordot(
        Ū2.conj().T[0,:].reshape(2,2),
        U2[:,0].reshape(2,2), (1,1))

#################################
# Aprrox Environment Calc
#################################
approx_r = approx_env(U2, U2)
print("Approximate Environment with U2:\n")
print(np.round(approx_r, 3))
print("Norm: ", np.linalg.norm(approx_r))
print("\nRenormalized:\n")
print(np.round(approx_r / np.linalg.norm(approx_r),3))
#################################

def approx_adjusted_env(U2, U1, Ū2, Ū1):
    alt_state = alternate_state(U2, U1)
    alt_state_bar = alternate_state(Ū2, Ū1)
    tm = single_transfer_matrix(alt_state, alt_state_bar)
    η, v = eig(tm)
    vr = v[:,np.argmax(np.abs(η))].reshape(2,2)
    
    env = np.einsum(
        U2.conj().T[0,:].reshape(2,2),[0,1],
        vr,[1,3],
        U2[:,0].reshape(2,2),[2,3],
        )
    
    return env

#################################
# Approx Adjusted Env
#################################
approx_adj_r = approx_adjusted_env(U2, U1, U2, U1)
print("\nApprox Adjusted Env:\n")
print(np.round(approx_adj_r,3))
print("Norm: ", np.linalg.norm(approx_adj_r))
print("\nRenormalized:\n")
print(np.round(approx_adj_r / np.linalg.norm(approx_adj_r),3))
#################################

def exact_left_environment(s, s̄):
    tm = single_transfer_matrix(s,s̄)
    η, v = eig(tm.T)
    vl = v[:,np.argmax(np.abs(η))].reshape(2,2)
    return vl

#################################
# Left Environment
#################################

env_l = exact_left_environment(s,s)
print("\nLeft Environment:\n")
print(np.round(env_l,3))

Exact environment:

[[ 0.646+0.j    -0.11 -0.438j]
 [-0.11 +0.438j  0.418+0.j   ]]
Norm:  0.9999999999999999


Approximate Environment with U2:

[[ 0.607+0.j    -0.103-0.412j]
 [-0.103+0.412j  0.393+0.j   ]]
Norm:  0.9399585248669571

Renormalized:

[[ 0.646+0.j    -0.11 -0.438j]
 [-0.11 +0.438j  0.418+0.j   ]]

Approx Adjusted Env:

[[ 0.429+0.j    -0.073-0.291j]
 [-0.073+0.291j  0.278+0.j   ]]
Norm:  0.6646510469675294

Renormalized:

[[ 0.646+0.j    -0.11 -0.438j]
 [-0.11 +0.438j  0.418+0.j   ]]

Left Environment:

[[ 0.707+0.j -0.   +0.j]
 [-0.   -0.j  0.707+0.j]]


In [307]:
init_param = np.random.rand(30)
U2, U1 = U4(init_param[:15]), U4(init_param[15:])
s = state(U2,U1)

Ut = expm(-1j * 0.00 * Hamiltonian(1,1))

def obj(param, s, Ut):
    ss = chain(s,s).reshape(2,16,2)
    
    Ū2, Ū1 = U4(param[:15]), U4(param[15:])
    s̄ = state(Ū2,Ū1)
    
    s̄s̄ = chain(s̄,s̄).reshape(2,16,2)
    
    exact_r = exact_right_environment(s,s̄)
    # exact_r = approx_env(U2, Ū2)
    # exact_r = approx_adjusted_env(U2, U1, Ū2, Ū1)
    exact_l = exact_left_environment(s,s̄)
    
    overlap = np.einsum(
        s̄s̄.conj(), [3,1,5],
        Ut,        [1,4],
        exact_r,   [5,2],
        exact_l,   [3,0],
        ss,        [0,4,2]
    )
    
    return -np.abs(overlap)

res = minimize(
    obj,
    x0 = init_param,
    method = "Nelder-Mead",
    args = (s, Ut)
)
print("Original: ", obj(init_param, s, Ut))
print("Optimized: ", res.fun)

Original:  -0.7564956001370454
Optimized:  -0.9322555089864284


In [294]:
res.x - init_param

array([-0.07800775,  0.23163819,  0.64348245, -0.01022623, -0.12585477,
        0.9366325 ,  0.12059929, -0.17741952, -0.40058811, -0.61839636,
        0.00236051,  0.14295287, -0.22094584,  0.29116078,  0.06966903,
        0.02042086,  0.14549394, -0.22038156, -0.81845872, -0.14689175,
       -0.57151976, -0.7151446 , -0.1592956 , -0.55012741, -1.04393014,
        0.26655502,  0.19117144,  0.09925475,  0.24383599,  0.27959986])

In [301]:
obj(init_param, s, Ut)

-0.5